In [175]:
# This code extracts the final trading price for Winner, Loser, Draw in each market

import json
import os
from os import path
import glob
import pandas as pd
import numpy as np
import timeit

#### NOTES ####
# Keys per file:

# pt = published Time (in millis since epoch) (int)
# mc = market change, changes to market prices, runners or market definition - can be disregarded
# op = operation type (string)
# clk = sequence token - *can be disregarded as part of this data set* (string)

# https://historicdata.betfair.com/Betfair-Historical-Data-Feed-Specification.pdf


In [176]:
# Get event and market information
def get_event_info(mkinfo, mkl):
    
    for i in range(0, mkl):
        mlist = mkinfo[i]['mc']
        keys = ['eventId', 'eventName', 'marketTime', 'countryCode', 'timezone', 'settledTime']
        rkeys = ['id', 'name', 'status']
        for d in mlist:  # gets id, marketDefinition as keys if exists
            md = 'marketDefinition'
            mid = (d.get('id'))
            if md in str(d.keys()):
                if d['marketDefinition'].get('status') == 'CLOSED':
                    mklist = [d['marketDefinition'].get(key) for key in keys]  # general market and event info
                    tm1 = [d['marketDefinition'].get('runners')[0].get(key) for key in rkeys]  # first team
                    tm2 = [d['marketDefinition'].get('runners')[1].get(key) for key in rkeys]  # second team
                    draw = [d['marketDefinition'].get('runners')[2].get(key) for key in rkeys]  # draw

    return(mid, mklist, tm1, tm2, draw)

In [177]:
# Get indexes of where market is inPlay = True/False
def get_inplay_indices(mkinfo, mkl):
    
    list_no = [] # inPlay = True indices
    list_nof = [] # inPlay = False indices
    
    for j in range(0, mkl):
        nlist = mkinfo[j]['mc']
        for d in nlist:
            md = 'marketDefinition'
            if md in str(d.keys()):
                # append indexes where inPlay status is true
                if ((d['marketDefinition'].get('status') == 'OPEN') and (
                    d['marketDefinition'].get('turnInPlayEnabled') == True) and (
                    d['marketDefinition'].get('inPlay') == True) and (
                    d['marketDefinition'].get('runners')[0].get('status') == 'ACTIVE')):
                    list_no.append(j)
                # append indexes where inPlay status is false
                if ((d['marketDefinition'].get('status') == 'OPEN') and (
                    d['marketDefinition'].get('inPlay') == False) and (
                    d['marketDefinition'].get('runners')[0].get('status') == 'ACTIVE')):
                    list_nof.append(j)
                    
    return(list_no, list_nof)

In [178]:
# Get ltps between last inPlay = False index and first inPlay = True
def get_final_ltps(list_no, list_nof, mkinfo):

    # Get all ltps between indexes
    rdf = pd.DataFrame()
    llist = mkinfo[:][(idx_lp + 1):idx_fp]  # get market info between last inPlay:False and first inPlay True
    lenlist = len(llist)


    for k in range(0, lenlist):
        mclist = llist[k].get('mc')  # gets mc list
        rclist = mclist[0].get('rc')  # gets rc list

        if rclist is None:
            pass

        else:

     # Select runner_id, and ltp
            for idx in rclist:
                akeys = ['id', 'ltp']
                ltpinfo = ([idx.get(key) for key in akeys]) # extracts id (W/L/D) and associated trading price


                flat = [ltpinfo[0], ltpinfo[1]]
                rdf = rdf.append([flat], ignore_index = True)

    if len(rdf) > 100:
        rdf = rdf.tail(100)
    
    try:
        # Get final ltps
        ltps = pd.DataFrame()
        rdf['idx'] = rdf.index
        ndf = rdf.groupby(rdf[0]).max()

        for idx in ndf.idx:
            rd = rdf.loc[rdf.idx == idx]
            rdx = [rd.iloc[0][0], rd.iloc[0][1], rd.iloc[0]['idx']]
            ltps = ltps.append([rdx], ignore_index = False)
   
    except:
        pass
    
    return(ltps, rdf, idx_fp, idx_lp)


In [179]:
def get_market_df(mid, ltps, mklist, tm1, tm2, idx_lp, idx_fp, mkl, file_var, filename):
    
    market_df = pd.DataFrame()
    
    # Append winner team info

    if len(ltps.loc[ltps[0] == tm1[0], 1]) == 0:
        pass

    else:

        markif_win = pd.Series([mid, mklist[0], mklist[1], mklist[2], mklist[3], mklist[4], mklist[5], tm1[0], tm1[1], tm1[2], ltps.loc[ltps[0] == tm1[0], 0].item(), ltps.loc[ltps[0] == tm1[0], 1].item(), [idx_lp, idx_fp], mkl, file_var, filename],
                               ['market_id', 'event_id', 'event_name', 'market_time', 'country_code', 'timezone', 'suspend_time', 'team_id', 'team_name', 'team_status', 'result_id', 'ltp', 'idx_in_file', 'file_len', 'file_no', 'file_name'])
        market_df = market_df.append([markif_win], ignore_index = True)

    # Append loser team info

    if len(ltps.loc[ltps[0] == tm2[0], 1]) == 0:
        pass

    else:

        markif_lost = pd.Series([mid, mklist[0], mklist[1], mklist[2], mklist[3], mklist[4], mklist[5], tm2[0], tm2[1], tm2[2], ltps.loc[ltps[0] == tm2[0], 0].item(), ltps.loc[ltps[0] == tm2[0], 1].item(), [idx_lp, idx_fp], mkl, file_var, filename],
                                ['market_id', 'event_id', 'event_name', 'market_time', 'country_code', 'timezone', 'suspend_time', 'team_id', 'team_name', 'team_status', 'result_id', 'ltp', 'idx_in_file', 'file_len', 'file_no', 'file_name'])
        market_df = market_df.append([markif_lost], ignore_index = True)

    # Append draw info

    # Check if draw exists
    if len(ltps.loc[ltps[0] == draw[0], 1]) == 0:
        pass

    else:

        markif_draw = pd.Series([mid, mklist[0], mklist[1], mklist[2], mklist[3], mklist[4], mklist[5], draw[0], draw[1], draw[2], ltps.loc[ltps[0] == draw[0], 0].item(), ltps.loc[ltps[0] == draw[0], 1].item(), [idx_lp, idx_fp], mkl, file_var, filename],
                                ['market_id', 'event_id', 'event_name', 'market_time', 'country_code', 'timezone', 'suspend_time','team_id', 'team_name', 'team_status', 'result_id', 'ltp', 'idx_in_file', 'file_len', 'file_no', 'file_name'])
        market_df = market_df.append([markif_draw], ignore_index = True)

    market_df = market_df[market_df.team_id == market_df.result_id]  # drop rows with mismatched ids
    market_df['prob'] = 1. / market_df.ltp
    
    return market_df

In [180]:
# Import json files in directory

#dir = 'soccerdata/'
#filelist = [dir +file for file in os.listdir(dir) if os.path.getsize(dir +file) > 11000]
#print(len(filelist))

filelist = ['BASIC-1.128923161.json']

# Setup dataframe to collect information

# Create csv file to append information to and open here:
#with open('all_market_data.csv', 'a') as f:

file_var = 0

for file in filelist:

    # try:
    #     filedata = open(dir +file, encoding='latin_1').read()

    # except:
    #     filedata = open(dir +file, encoding='utf_8').read()

    # Open multi-line file and get info
    mkinfo = []
    for line in open(file, 'r'):
        mkinfo.append(json.loads(line))

    # File info
    mkl = len(mkinfo)
    filename = os.path.basename(file)
    file_var += 1

    print('Get event and market information')
    mid, mklist, tm1, tm2, draw = get_event_info(mkinfo, mkl)

    print('Get indexes of where market is inPlay = True/False')
    list_no, list_nof = get_inplay_indices(mkinfo, mkl)
    
    if list_no and list_nof and list_no != [0] and list_nof != [0]:
        idx_fp = min(list_no)  # first index where inPlay is True (+1 to include in range)
        idx_lp = max(list_nof)  # last index where inPlay is False
    
        print([idx_lp, idx_fp])
  
        # Check indexes make sense and last inPlay is False is before first inPlay is True
        print('Get final trading prices')
        if (idx_lp > idx_fp) or ((idx_lp + 1) == idx_fp) or ((idx_fp + 1) == idx_lp):  # check indexes
            pass

        else:
            ltps, rdf, idx_fp, idx_lp = get_final_ltps(list_no, list_nof, mkinfo)
            
            try:
                
                print('Append information and get market_df')
                market_df = get_market_df(mid, ltps, mklist, tm1, tm2, idx_lp, idx_fp, mkl, file_var, filename)

                # Check probabilities add up to 1 
                if 0.98 <= sum(market_df.prob) <= 1.015:
                    #market_df.to_csv(f, header=False)  # Append to existing csv
                    all_mkt_dfs = all_mkt_dfs.append(market_df)
                    print('file number {} processed, name: {}'.format(file_var, filename))

                else:
                    print('probabilities did not add up to 1')
            
            except:
                pass


Get event and market information
Get indexes of where market is inPlay = True/False
[1, 30]
Get final trading prices
Append information and get market_df
file number 1 processed, name: BASIC-1.128923161.json


In [181]:
#print(timeit.timeit(json_extract(filelist)))
market_df

,market_id,event_id,event_name,market_time,country_code,timezone,suspend_time,team_id,team_name,team_status,result_id,ltp,idx_in_file,file_len,file_no,file_name,prob
0,1.128923161,28061387,Al Hussein v Al Faisaly (JOR),2016-12-31T13:00:00.000Z,JO,Europe/London,2016-12-31T14:58:12.000Z,5360907,Al Hussein,LOSER,5360907.0,3.80,"[1, 30]",106,1,BASIC-1.128923161.json,0.263158
1,1.128923161,28061387,Al Hussein v Al Faisaly (JOR),2016-12-31T13:00:00.000Z,JO,Europe/London,2016-12-31T14:58:12.000Z,5551580,Al Faisaly (JOR),WINNER,5551580.0,2.46,"[1, 30]",106,1,BASIC-1.128923161.json,0.406504
2,1.128923161,28061387,Al Hussein v Al Faisaly (JOR),2016-12-31T13:00:00.000Z,JO,Europe/London,2016-12-31T14:58:12.000Z,58805,The Draw,LOSER,58805.0,3.00,"[1, 30]",106,1,BASIC-1.128923161.json,0.333333
